In [ ]:
!pip install Keras-Preprocessing

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.7 MB/s eta 0:00:00


In [ ]:
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Linear SVM

In [ ]:
# Importing required libraries
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.svm import LinearSVC

# Reading data into a pandas DataFrame
data = pd.read_csv (r'/content/drive/MyDrive/New/Clean/isot_clean.csv')

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_v = TfidfVectorizer(max_features=5000, ngram_range=(1,3))
X = tfidf_v.fit_transform(data['statement'].values.astype('U'))
y = data['label'].values

In [ ]:
import joblib

# Creating a Linear SVM classifier
svm = LinearSVC()

# Performing five-fold cross-validation
scores = cross_val_score(svm, X, y, cv=5)

# Printing the cross-validation scores
print("Cross-validation scores:", scores)

# Computing and printing accuracy, recall, precision, and F1 score
y_pred = cross_val_predict(svm, X, y, cv=5)
accuracy = accuracy_score(y, y_pred)
recall = recall_score(y, y_pred)
precision = precision_score(y, y_pred)
f1 = f1_score(y, y_pred)
print("Accuracy:", accuracy)
print("Recall:", recall)
print("Precision:", precision)
print("F1 score:", f1)

# Save the trained model to a file
joblib.dump(svm, '/content/drive/MyDrive/Colab Notebooks/weights/ISOT/linearsvc_isot.pkl')
 

Cross-validation scores: [0.98187982 0.9811784  0.98258125 0.98129311 0.98105928]
Accuracy: 0.9815983913206136
Recall: 0.9816726060998751
Precision: 0.981899824090362
F1 score: 0.9817862019486682


['/content/drive/MyDrive/Colab Notebooks/weights/ISOT/linearsvc_isot.pkl']

# LSTM


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

df = pd.read_csv (r'/content/drive/MyDrive/New/Clean/isot_clean.csv')

# Tokenize the text
# Split data into statements and labels
statements = df['statement'].astype(str)
labels = df['label']

# Tokenize statements
tokenizer = Tokenizer()
tokenizer.fit_on_texts(statements)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(statements)

# Pad sequences
Max_Len = 100
padded_sequences = pad_sequences(sequences, maxlen=Max_Len)


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, df['label'].values, test_size=0.2, random_state=0)

# Build the model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=5000, output_dim=32, input_length=Max_Len),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), verbose=2)

# Evaluate the model
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5).astype(int)

# Calculate the evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1)

# Save the model
model.save('/content/drive/MyDrive/Colab Notebooks/weights/ISOT/lstm_isot.h5')

# Load the saved model
# loaded_model = tf.keras.models.load_model('lstm_fn.h5')


Epoch 1/10
1070/1070 - 61s - loss: 0.1631 - accuracy: 0.9412 - val_loss: 0.0974 - val_accuracy: 0.9653 - 61s/epoch - 57ms/step
Epoch 2/10
1070/1070 - 13s - loss: 0.0703 - accuracy: 0.9767 - val_loss: 0.0898 - val_accuracy: 0.9682 - 13s/epoch - 12ms/step
Epoch 3/10
1070/1070 - 9s - loss: 0.0478 - accuracy: 0.9847 - val_loss: 0.1023 - val_accuracy: 0.9693 - 9s/epoch - 9ms/step
Epoch 4/10
1070/1070 - 9s - loss: 0.0389 - accuracy: 0.9874 - val_loss: 0.1051 - val_accuracy: 0.9703 - 9s/epoch - 8ms/step
Epoch 5/10
1070/1070 - 9s - loss: 0.0237 - accuracy: 0.9927 - val_loss: 0.0984 - val_accuracy: 0.9739 - 9s/epoch - 8ms/step
Epoch 6/10
1070/1070 - 9s - loss: 0.0195 - accuracy: 0.9939 - val_loss: 0.1051 - val_accuracy: 0.9694 - 9s/epoch - 8ms/step
Epoch 7/10
1070/1070 - 8s - loss: 0.0158 - accuracy: 0.9951 - val_loss: 0.1179 - val_accuracy: 0.9697 - 8s/epoch - 8ms/step
Epoch 8/10
1070/1070 - 9s - loss: 0.0088 - accuracy: 0.9977 - val_loss: 0.1674 - val_accuracy: 0.9384 - 9s/epoch - 8ms/step
Ep

# BI LSTM


In [ ]:
import numpy as np
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, Bidirectional
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load the data from CSV file
data = pd.read_csv('/content/drive/MyDrive/New/Clean/isot_clean.csv')

# Split data into statements and labels
statements = data['statement']
labels = data['label']

# Tokenize statements
tokenizer = Tokenizer()
tokenizer.fit_on_texts(statements)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(statements)

# Pad sequences
max_len = 100
padded_sequences = pad_sequences(sequences, maxlen=max_len)

# Split data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(padded_sequences, labels, test_size=0.2)

# Define model architecture
model = Sequential()
model.add(Embedding(len(word_index) + 1, 128, input_length=max_len))
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(Bidirectional(LSTM(32)))
model.add(Dense(1, activation='sigmoid'))

# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

# Train model
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test))

# Evaluate the model
y_pred = model.predict(x_test)
y_pred = (y_pred > 0.5).astype(int)

# Calculate the evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1)


# Save the model
model.save('/content/drive/MyDrive/Colab Notebooks/weights/ISOT/bilstm_isot.h5')

# Load the saved model
# loaded_model = tf.keras.models.load_model('lstm_fn.h5')


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 100, 128)          3463680   
                                                                 
 bidirectional (Bidirectiona  (None, 100, 128)         98816     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 64)               41216     
 nal)                                                            
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 3,603,777
Trainable params: 3,603,777
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
1070/1070 [=================

# HYBRID


In [ ]:
import numpy as np
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.layers import Embedding, Conv1D, MaxPooling1D, Bidirectional, LSTM, Dense, Dropout
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Preprocessing
MAX_NB_WORDS = 50000  # Maximum number of words to be used in the tokenizer
MAX_SEQUENCE_LENGTH = 300  # Maximum length of each news statement
EMBEDDING_DIM = 100  # Dimension of the word embedding
VALIDATION_SPLIT = 0.2  # Percentage of data to use for validation
BATCH_SIZE = 128
EPOCHS = 10

# Load the data from CSV file
df = pd.read_csv('/content/drive/MyDrive/New/Clean/isot_clean.csv')

# Split data into statements and labels
statements = df['statement'].astype(str)
labels = df['label']

# Tokenize statements
tokenizer = Tokenizer()
tokenizer.fit_on_texts(statements)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(statements)

# Pad sequences
Max_Len = max([len(x) for x in statements])
padded_sequences = pad_sequences(sequences, maxlen=Max_Len)

print('Shape of data tensor:', padded_sequences.shape)

# Define the maximum number of words to keep in the vocabulary
MAX_NUM_WORDS = 20000

# Convert the labels to one-hot encoding
y = df['label']
print('Shape of label tensor:', y.shape)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, df['label'].values, test_size=0.2)

# Build the model
model = Sequential()
model.add(Embedding(input_dim=MAX_NB_WORDS, output_dim=EMBEDDING_DIM, input_length=Max_Len))
model.add(Conv1D(128, 5, activation='relu'))
model.add(Dropout(0.3))
model.add(MaxPooling1D(2))
model.add(Conv1D(128, 5, activation='relu'))
model.add(MaxPooling1D(2))
model.add(Bidirectional(LSTM(32)))

model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

# Train the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=128)

# Evaluate the model
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5).astype(int)

# Calculate the evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1)

# Save the model
model.save('/content/drive/MyDrive/Colab Notebooks/weights/ISOT/hybrid_isot.h5')

# Load the saved model
# loaded_model = tf.keras.models.load_model('lstm_fn.h5')


Shape of data tensor: (42768, 29569)
Shape of label tensor: (42768,)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 29569, 100)        5000000   
                                                                 
 conv1d (Conv1D)             (None, 29565, 128)        64128     
                                                                 
 dropout (Dropout)           (None, 29565, 128)        0         
                                                                 
 max_pooling1d (MaxPooling1D  (None, 14782, 128)       0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 14778, 128)        82048     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 7389, 128)      

# GLOVE HYBRID


In [ ]:
import numpy as np
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.layers import Embedding, Conv1D, MaxPooling1D, Bidirectional, LSTM, Dense, Dropout
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load the data from CSV file
df = pd.read_csv('/content/drive/MyDrive/New/Clean/isot_clean.csv')

MAX_NB_WORDS = 50000  # Maximum number of words to be used in the tokenizer
VALIDATION_SPLIT = 0.2

# Split data into statements and labels
statements = df['statement'].astype(str)
labels = df['label']

# Tokenize statements
tokenizer = Tokenizer()
tokenizer.fit_on_texts(statements)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(statements)

# Pad sequences
Max_Len = max([len(x) for x in statements])
padded_sequences = pad_sequences(sequences, maxlen=Max_Len)

print('Shape of data tensor:', padded_sequences.shape)

# Define the maximum number of words to keep in the vocabulary
MAX_NUM_WORDS = 20000

# Convert the labels to one-hot encoding
y = df['label']
print('Shape of label tensor:', y.shape)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, df['label'].values, test_size=0.2)


# Load the GloVe embeddings
embeddings_index = {}
f = open('/content/drive/MyDrive/Colab Notebooks/weights/glove.6B.300d.txt', encoding='utf8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

# Create an embedding matrix
embedding_dim = 300
word_index = tokenizer.word_index
num_words = min(MAX_NUM_WORDS, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, embedding_dim))
for word, i in word_index.items():
    if i >= MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

# Build the model
model = Sequential()
model.add(Embedding(input_dim=num_words, output_dim=embedding_dim, input_length=Max_Len, weights=[embedding_matrix], trainable=False))
model.add(Conv1D(128, 5, activation='relu'))
model.add(Dropout(0.3))
model.add(MaxPooling1D(2))
model.add(Conv1D(128, 5, activation='relu'))
model.add(MaxPooling1D(2))
model.add(Bidirectional(LSTM(32)))

model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=128)

# Evaluate the model
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5).astype(int)

# Calculate the evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1)

# Save the model
model.save('/content/drive/MyDrive/Colab Notebooks/weights/ISOT/glove6b300_isot.h5')

# Load the saved model
# loaded_model = tf.keras.models.load_model('lstm_fn.h5')


Shape of data tensor: (42768, 29569)
Shape of label tensor: (42768,)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 29569, 300)        6000000   
                                                                 
 conv1d (Conv1D)             (None, 29565, 128)        192128    
                                                                 
 dropout (Dropout)           (None, 29565, 128)        0         
                                                                 
 max_pooling1d (MaxPooling1D  (None, 14782, 128)       0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 14778, 128)        82048     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 7389, 128)      

In [ ]:
0.0456import numpy as np
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.layers import Embedding, Conv1D, MaxPooling1D, Bidirectional, LSTM, Dense, Dropout
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load the data from CSV file
df = pd.read_csv('/content/drive/MyDrive/New/Clean/isot_clean.csv')

MAX_NB_WORDS = 50000  # Maximum number of words to be used in the tokenizer
VALIDATION_SPLIT = 0.2

# Split data into statements and labels
statements = df['statement'].astype(str)
labels = df['label']

# Tokenize statements
tokenizer = Tokenizer()
tokenizer.fit_on_texts(statements)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(statements)

# Pad sequences
Max_Len = max([len(x) for x in statements])
padded_sequences = pad_sequences(sequences, maxlen=Max_Len)

print('Shape of data tensor:', padded_sequences.shape)

# Define the maximum number of words to keep in the vocabulary
MAX_NUM_WORDS = 20000

# Convert the labels to one-hot encoding
y = df['label']
print('Shape of label tensor:', y.shape)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, df['label'].values, test_size=0.2)

# Load the GloVe embeddings
embeddings_index = {}
f = open('/content/drive/MyDrive/Colab Notebooks/weights/glove.42B.300d.txt', encoding='utf8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

# Create an embedding matrix
embedding_dim = 300
word_index = tokenizer.word_index
num_words = min(MAX_NUM_WORDS, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, embedding_dim))
for word, i in word_index.items():
    if i >= MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

# Build the model
model = Sequential()
model.add(Embedding(input_dim=num_words, output_dim=embedding_dim, input_length=Max_Len, weights=[embedding_matrix], trainable=False))
model.add(Conv1D(128, 5, activation='relu'))
model.add(Dropout(0.3))
model.add(MaxPooling1D(2))
model.add(Conv1D(128, 5, activation='relu'))
model.add(MaxPooling1D(2))
model.add(Bidirectional(LSTM(32)))

model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=128)

# Evaluate the model
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5).astype(int)

# Calculate the evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1)

# Save the model
model.save('/content/drive/MyDrive/Colab Notebooks/weights/ISOT/glove42b_isot.h5')

# Load the saved model
# loaded_model = tf.keras.models.load_model('lstm_fn.h5')


Shape of data tensor: (42768, 29569)
Shape of label tensor: (42768,)
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 29569, 300)        6000000   
                                                                 
 conv1d_2 (Conv1D)           (None, 29565, 128)        192128    
                                                                 
 dropout_1 (Dropout)         (None, 29565, 128)        0         
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 14782, 128)       0         
 1D)                                                             
                                                                 
 conv1d_3 (Conv1D)           (None, 14778, 128)        82048     
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 7389, 128)    

# Word2Vec

In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, Conv1D, MaxPooling1D, Bidirectional, LSTM, Dense, Dropout
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


df = pd.read_csv('/content/drive/MyDrive/New/Clean/isot_clean.csv')

MAX_NB_WORDS = 50000  # Maximum number of words to be used in the tokenizer
VALIDATION_SPLIT = 0.2

# Tokenize the news statements
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(df['statement'])
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

# Convert the news statements to sequences of integers
sequences = tokenizer.texts_to_sequences(df['statement'])

# Pad the sequences
Max_Len = max([len(x) for x in df['statement']])
padded = pad_sequences(sequences, maxlen=Max_Len)

print('Shape of data tensor:', padded.shape)

# Define the maximum number of words to keep in the vocabulary
MAX_NUM_WORDS = 20000

# Convert the labels to one-hot encoding
y = df['label']
print('Shape of label tensor:', y.shape)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(padded, df['label'].values, test_size=0.2)

# Load the Word2Vec embeddings
from gensim.models.word2vec import Word2Vec
import gensim.downloader as api

# Load the pre-trained Word2Vec model
w2v_model = api.load('word2vec-google-news-300')

# Create an embedding matrix
embedding_dim = 300
word_index = tokenizer.word_index
num_words = min(MAX_NUM_WORDS, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, embedding_dim))
for word, i in word_index.items():
    if i >= MAX_NUM_WORDS:
        continue
    if word in w2v_model:
        embedding_vector = w2v_model[word]
        embedding_matrix[i] = embedding_vector

# Build the model
model = Sequential()
model.add(Embedding(input_dim=num_words, output_dim=embedding_dim, input_length=Max_Len, weights=[embedding_matrix], trainable=False))
model.add(Conv1D(128, 5, activation='relu'))
model.add(Dropout(0.3))
model.add(MaxPooling1D(2))
model.add(Conv1D(128, 5, activation='relu'))
model.add(MaxPooling1D(2))
model.add(Bidirectional(LSTM(32)))

model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=128)

# Evaluate the model
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5).astype(int)

# Calculate the evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1)

# Save the model
model.save('/content/drive/MyDrive/Colab Notebooks/weights/ISOT/hybrid_word2vec_isot.h5')
# Load the saved model
# loaded_model = tf.keras.models.load_model('lstm_fn.h5')

Found 27059 unique tokens.
Shape of data tensor: (42768, 29569)
Shape of label tensor: (42768,)
[==================================================] 100.0% 1662.8/1662.8MB downloaded
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 29569, 300)        6000000   
                                                                 
 conv1d_2 (Conv1D)           (None, 29565, 128)        192128    
                                                                 
 dropout_1 (Dropout)         (None, 29565, 128)        0         
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 14782, 128)       0         
 1D)                                                             
                                                                 
 conv1d_3 (Conv1D)           (None, 14778, 128)        82048     
   

# fastText

In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, Conv1D, MaxPooling1D, Bidirectional, LSTM, Dense, Dropout
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load the data from CSV file
df = pd.read_csv('/content/drive/MyDrive/New/Clean/isot_clean.csv')

MAX_NB_WORDS = 50000  # Maximum number of words to be used in the tokenizer
VALIDATION_SPLIT = 0.2

# Split data into statements and labels
statements = df['statement'].astype(str)
labels = df['label']

# Tokenize statements
tokenizer = Tokenizer()
tokenizer.fit_on_texts(statements)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(statements)

# Pad sequences
Max_Len = max([len(x) for x in statements])
padded_sequences = pad_sequences(sequences, maxlen=Max_Len)

print('Shape of data tensor:', padded_sequences.shape)

# Define the maximum number of words to keep in the vocabulary
MAX_NUM_WORDS = 20000

# Convert the labels to one-hot encoding
y = df['label']
print('Shape of label tensor:', y.shape)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, df['label'].values, test_size=0.2)

# Load the Word2Vec embeddings
from gensim.models.word2vec import Word2Vec
import gensim.downloader as api

# Load the pre-trained Word2Vec model
ft_model = api.load('fasttext-wiki-news-subwords-300')

# Create an embedding matrix
embedding_dim = 300
word_index = tokenizer.word_index
num_words = min(MAX_NUM_WORDS, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, embedding_dim))
for word, i in word_index.items():
    if i >= MAX_NUM_WORDS:
        continue
    if word in ft_model:
        embedding_vector = ft_model[word]
        embedding_matrix[i] = embedding_vector

# Build the model
model = Sequential()
model.add(Embedding(input_dim=num_words, output_dim=embedding_dim, input_length=Max_Len, weights=[embedding_matrix], trainable=False))
model.add(Conv1D(128, 5, activation='relu'))
model.add(Dropout(0.3))
model.add(MaxPooling1D(2))
model.add(Conv1D(128, 5, activation='relu'))
model.add(MaxPooling1D(2))
model.add(Bidirectional(LSTM(32)))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=128)

# Evaluate the model
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5).astype(int)

# Calculate the evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1)

# Save the model
model.save('/content/drive/MyDrive/Colab Notebooks/weights/ISOT/ft_isot.h5')

# Load the saved model
# loaded_model = tf.keras.models.load_model('lstm_fn.h5')

Shape of data tensor: (42768, 29569)
Shape of label tensor: (42768,)
[==================================================] 100.0% 958.5/958.4MB downloaded
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 29569, 300)        6000000   
                                                                 
 conv1d (Conv1D)             (None, 29565, 128)        192128    
                                                                 
 dropout (Dropout)           (None, 29565, 128)        0         
                                                                 
 max_pooling1d (MaxPooling1D  (None, 14782, 128)       0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 14778, 128)        82048     
                                  